# Feature Selection: SVM + RFE
```
1. 用SVM來作特徵選取看來也是不適用在文章分類上, 因為也敵不過 TF-IDF, TextRank
2. 大家頗看好 L1-SVM feature selection
3. 這裡研究不深... 目前只知道預測結果是category的可以用 SVC, LinearSVC C結尾的, 預測結果是連續變數的(像房價範圍預估), 就必須用SVR, LinearSVR 回歸系列的libs
3. 參考資料: http://scikit-learn.org/stable/modules/feature_selection.html#feature-selection-using-selectfrommodel
            https://machine-learning-python.kspax.io/Feature_Selection/intro.html
```

In [8]:
%reset -f
%pylab inline
import sys, codecs, pandas as pd, numpy as np, os, re, traceback, pprint, sqlite3, importlib
# import jieba, jieba.analyse
from __future__ import division, unicode_literals, with_statement, print_function
from sklearn import datasets
from contextlib import closing


class DataSources(object):    
    def sklearnDs(self):
        return sqlite3.connect('sklearnDs.sqlite')

ds = DataSources()

Populating the interactive namespace from numpy and matplotlib


In [55]:
# 改用波士頓房價的屬性萃取重要feature
import pandas as pd

"""
1. CRIM: per capita crime rate by town 
2. ZN: proportion of residential land zoned for lots over 25,000 sq.ft. 
3. INDUS: proportion of non-retail business acres per town 
4. CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise) 
5. NOX: nitric oxides concentration (parts per 10 million) 
6. RM: average number of rooms per dwelling 
7. AGE: proportion of owner-occupied units built prior to 1940 
8. DIS: weighted distances to five Boston employment centres 
9. RAD: index of accessibility to radial highways 
10. TAX: full-value property-tax rate per $10,000 
11. PTRATIO: pupil-teacher ratio by town 
12. B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town 
13. LSTAT: % lower status of the population 
14. MEDV: Median value of owner-occupied homes in $1000's
"""
from sklearn.datasets import load_boston
boston = load_boston()
bostonDf = pd.DataFrame(data=boston.data, columns=boston.feature_names)
bostonDf['房價'] = boston.target
bostonDf.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,房價
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
5,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21,28.7
6,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43,22.9
7,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311,15.2,396.90,19.15,27.1
8,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311,15.2,386.63,29.93,16.5
9,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.10,18.9


In [56]:
# RFE: 遞回的 features 剃除, 怎麼地回的當然不太知道...

from sklearn.svm import LinearSVC, SVR, LinearSVR
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
 
boston = load_boston()
X = boston["data"]
Y = boston["target"]
names = boston["feature_names"]
 
# 這裡不能用SVC, 因為 lable y是個連續的數字, 
svr = SVR(kernel='linear')
#rank all features, i.e continue the elimination until the last one
rfe = RFE(estimator=svr, n_features_to_select=5, step=1)
rfe.fit(X, Y)
 
print("Features sorted by their rank:")
# r = sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), names))
# dict(r)
rfe.ranking_

Features sorted by their rank:


array([3, 5, 4, 1, 1, 1, 6, 2, 8, 9, 1, 7, 1])

In [53]:
Y

array([ 24. ,  21.6,  34.7,  33.4,  36.2,  28.7,  22.9,  27.1,  16.5,
        18.9,  15. ,  18.9,  21.7,  20.4,  18.2,  19.9,  23.1,  17.5,
        20.2,  18.2,  13.6,  19.6,  15.2,  14.5,  15.6,  13.9,  16.6,
        14.8,  18.4,  21. ,  12.7,  14.5,  13.2,  13.1,  13.5,  18.9,
        20. ,  21. ,  24.7,  30.8,  34.9,  26.6,  25.3,  24.7,  21.2,
        19.3,  20. ,  16.6,  14.4,  19.4,  19.7,  20.5,  25. ,  23.4,
        18.9,  35.4,  24.7,  31.6,  23.3,  19.6,  18.7,  16. ,  22.2,
        25. ,  33. ,  23.5,  19.4,  22. ,  17.4,  20.9,  24.2,  21.7,
        22.8,  23.4,  24.1,  21.4,  20. ,  20.8,  21.2,  20.3,  28. ,
        23.9,  24.8,  22.9,  23.9,  26.6,  22.5,  22.2,  23.6,  28.7,
        22.6,  22. ,  22.9,  25. ,  20.6,  28.4,  21.4,  38.7,  43.8,
        33.2,  27.5,  26.5,  18.6,  19.3,  20.1,  19.5,  19.5,  20.4,
        19.8,  19.4,  21.7,  22.8,  18.8,  18.7,  18.5,  18.3,  21.2,
        19.2,  20.4,  19.3,  22. ,  20.3,  20.5,  17.3,  18.8,  21.4,
        15.7,  16.2,

In [62]:
# 前五名重要的feature
"""
1. CRIM: per capita crime rate by town 
2. ZN: proportion of residential land zoned for lots over 25,000 sq.ft. 
3. INDUS: proportion of non-retail business acres per town 
4. CHAS: PCA dummy variable (= 1 if tract bounds river; 0 otherwise) 
5. NOX: nitric oxides concentration (parts per 10 million) 
6. RM: average number of rooms per dwelling 
7. AGE: proportion of owner-occupied units built prior to 1940 
8. DIS: weighted distances to five Boston employment centres 
9. RAD: index of accessibility to radial highways 
10. TAX: full-value property-tax rate per $10,000 
11. PTRATIO: pupil-teacher ratio by town 
12. B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town 
13. LSTAT: % lower status of the population 
14. MEDV: Median value of owner-occupied homes in $1000's
"""
bostonDf[boston.feature_names[rfe.ranking_ == 1].tolist() + ['房價']].sort_values('房價', ascending=False)

,CHAS,NOX,RM,PTRATIO,LSTAT,房價
283,1,0.4010,7.923,13.6,3.16,50.0
225,0,0.5040,8.725,17.4,4.63,50.0
369,1,0.6310,6.683,20.2,3.73,50.0
370,1,0.6310,7.016,20.2,2.96,50.0
371,0,0.6310,6.216,20.2,9.53,50.0
372,1,0.6680,5.875,20.2,8.88,50.0
186,0,0.4880,7.831,17.8,4.45,50.0
204,0,0.4161,8.034,14.7,2.88,50.0
257,0,0.6470,8.704,13.0,5.12,50.0
195,0,0.4220,7.875,14.4,2.97,50.0


In [81]:
# 官方網站鳶尾花原本4維, 用LinearSVC降了一個維度(最後一維)

from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
iris = load_iris()
X, y = iris.data, iris.target
X.shape

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
print(X_new.shape)
model.get_support()

(150L, 3L)


array([ True,  True,  True, False], dtype=bool)

In [76]:
irisDf = pd.DataFrame(columns=[iris.feature_names], data=iris.data)
irisDf['target'] = iris.target_names[iris.target]
irisDf

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5.0,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
9,4.9,3.1,1.5,0.1,setosa
